# Efficiently train Large Language Models with LoRA and Hugging Face

In this blog, we are going to show you how to apply [Low-Rank Adaptation of Large Language Models (LoRA)](https://arxiv.org/abs/2106.09685) to fine-tune FLAN-T5 XXL (11 billion parameters) on a single GPU. We are going to leverage Hugging Face [Transformers](https://huggingface.co/docs/transformers/index), [Accelerate](https://huggingface.co/docs/accelerate/index), and [PEFT](https://github.com/huggingface/peft). 

You will learn how to:

1. Setup Development Environment
2. Load and prepare the dataset
3. Fine-Tune T5 with LoRA and bnb int-8
4. Evaluate & run Inference with LoRA FLAN-T5
5. Cost performance comparison

### Quick intro: PEFT or Parameter Efficient Fine-tunin

[PEFT](https://github.com/huggingface/peft), or Parameter Efficient Fine-tuning, is a new open-source library from Hugging Face to enable efficient adaptation of pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model's parameters. PEFT currently includes techniques for:

- LoRA: [LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- Prefix Tuning: [P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/pdf/2110.07602.pdf)
- P-Tuning: [GPT Understands, Too](https://arxiv.org/pdf/2103.10385.pdf)
- Prompt Tuning: [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/pdf/2104.08691.pdf)

*Note: This tutorial was created and run on a g5.2xlarge AWS EC2 Instance, including 1 NVIDIA A10G.*

## 1. Setup Development Environment

In our example, we use the [PyTorch Deep Learning AMI](https://docs.aws.amazon.com/dlami/latest/devguide/tutorial-pytorch.html) with already set up CUDA drivers and PyTorch installed. We still have to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages.

In [1]:
import numpy as np
from scipy.special import softmax
import pdb
import pandas as pd
import math
from typing import List
import random
import argparse
import torch


def sent_scoring(model_tokenizer, text, cuda, score_type="loss", output_attentions=False, length_normalize=False):
    model = model_tokenizer[0]
    tokenizer = model_tokenizer[1]
    assert model is not None
    assert tokenizer is not None
    encoded_text = tokenizer.encode(text)
    input_ids = torch.tensor(encoded_text).unsqueeze(0)
    if cuda:
        input_ids = input_ids.to('cuda')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids, output_attentions=output_attentions)
    loss, logits = outputs[:2]

    sentence_prob = loss.item()
    if score_type == "prob":
        if length_normalize:
            mult = 2
        else:
            mult = len(encoded_text)

        sentence_prob = math.exp(-1.0 * loss * (mult - 1))

    if output_attentions:
        attn = outputs["attentions"]
        return sentence_prob, attn, input_ids

    return sentence_prob

def confusion_matrix(P_forward_1, P_forward_2, P_backward_1, P_backward_2):
    correct_forward = len(np.where(np.array(P_forward_1) >= 0.5)[0]) + len(np.where(np.array(P_forward_2) >=0.5)[0])
    wrong_forward = len(P_forward_1) + len(P_forward_2) - correct_forward

    correct_backward = len(np.where(np.array(P_backward_1) >= 0.5)[0]) + len(np.where(np.array(P_backward_2) >=0.5)[0])
    wrong_backward = len(P_backward_1) + len(P_backward_2) - correct_backward

    print("correct forward", correct_forward, "wrong forward", wrong_forward, "correct backward", correct_backward, "wrong_backward", wrong_backward)

    results = {
        "correct_forward": correct_forward,
        "wrong_forward": wrong_forward,
        "correct_backward": correct_backward,
        "wrong_backward": wrong_backward
    }

    return results

from tqdm import tqdm

def evaluate_model(model, tokenizer, test_set, middle_phrase="", use_prefix=0, verbose=True, score_type="prob", use_cuda=False, return_acc=False, total = 1094) -> tuple:
    preds = []
    labels = []
    x_1 = []
    x_2 = []
    y_1 = []
    y_2 = []
    P_x_1 = []
    P_x_2 = []
    P_y_1 = []
    P_y_2 = []
    P_x_1_y_1 = []
    P_x_1_y_2 = []
    P_x_2_y_1 = []
    P_x_2_y_2 = []
    P_x_1_correct = []
    P_x_2_correct = []
    P_y_1_correct = []
    P_y_2_correct = []
    correct = 0

    for i, metaphor_data in tqdm(enumerate(test_set), total = total):
        ctx, p1, p2 = metaphor_data["startphrase"], metaphor_data["ending1"], metaphor_data["ending2"]
        labels.append(int(metaphor_data["labels"]))
        if use_prefix > 0:
            prefix_prompt = select_prefix_prompts(prompt_file, use_prefix) if use_prefix else ""
        else:
            prefix_prompt = ""

        sent1 = prefix_prompt + ctx + ". " + middle_phrase + p1 + "."
        sent2 = prefix_prompt + ctx + ". " + middle_phrase + p2 + "."

        score1 = sent_scoring((model, tokenizer), sent1, use_cuda, score_type=score_type)
        score2 = sent_scoring((model, tokenizer), sent2, use_cuda, score_type=score_type)

        if score_type == "loss":
            pred = 0 if score1 < score2 else 1
        else:
            pred = 1 if score1 < score2 else 0

        pred_sent = sent1 if pred == 0 else sent2

        if i % 2 == 0:
            x_1.append(ctx)
            x_1_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_1.append(x_1_score)
            y_1.append(p1)
            y_2.append(p2)
            y1_score = sent_scoring((model, tokenizer), p1 + ".", use_cuda, score_type=score_type)
            y2_score = sent_scoring((model, tokenizer), p2 + ".", use_cuda, score_type=score_type)
            P_y_1.append(y1_score)
            P_y_2.append(y2_score)

            P_x_1_y_1.append(score1)
            P_x_1_y_2.append(score2)
            P_x_1_correct.append(score1/(score1 + score2))

        else:
            x_2.append(ctx)
            x_2_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_2.append(x_2_score)
            P_x_2_y_1.append(score1)
            P_x_2_y_2.append(score2)
            P_x_2_correct.append(score2/(score1 + score2))

            P_y_1_correct.append(P_x_1_y_1[-1]/(P_x_1_y_1[-1] + score1))
            P_y_2_correct.append(score2/(P_x_1_y_2[-1] + score2))

        if verbose:
            print(f"Q: {ctx}: 1. {p1} 2. {p2}")
            print(f"model says '{pred_sent}' is more likely")
            print("\n")
        if pred == metaphor_data["labels"]:
            correct += 1
        preds.append(pred)

    cols = {"x_1": x_1, "x_2": x_2, "y_1": y_1, "y_2": y_2, "P(x_1)": P_x_1, "P(x_2)": P_x_2, "P(y_1)": P_y_1, "P(y_2)": P_y_2,
        "P(x_1, y_1)": P_x_1_y_1, "P(x_1, y_2)": P_x_1_y_2, "P(x_2, y_1)": P_x_2_y_1, "P(x_2, y_2)": P_x_2_y_2,
        "P(y_1|x_1)": P_x_1_correct, "P(y_2|x_2)": P_x_2_correct, "P(x_1|y_1)": P_y_1_correct, "P(x_2|y_2)": P_y_2_correct}
    out_df = pd.DataFrame(cols)

    if return_acc:
        return correct/len(preds), out_df, preds, labels

    return out_df, preds, labels

def compute_stats(total_df: pd.DataFrame, all_preds: List, all_labels: List) -> None:
    print("overall accuracy: ")
    accuracyy = len(np.where(np.array(all_preds) == np.array(all_labels))[0])/len(all_labels)
    print(accuracyy)
    print("confusion matrix: ")
    matrix_dic = confusion_matrix(list(total_df["P(y_1|x_1)"]), list(total_df["P(y_2|x_2)"]), list(total_df["P(x_1|y_1)"]), list(total_df["P(x_2|y_2)"]))

    return accuracyy, matrix_dic


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
!pip uninstall datasets -y
!pip install datasets

Found existing installation: datasets 2.1.0
Uninstalling datasets-2.1.0:
  Successfully uninstalled datasets-2.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 12.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.9.0
    Uninstalling fsspec-2023.9.0:
      Successfully uninstalled fsspec-2023.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is inco

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [4]:
# install Hugging Face Libraries
#!pip install "peft==0.2.0"
#!pip install "transformers==4.27.1" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr 

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=50f0045b726343cae3f2bc488faca8122d94b8d369278df49c838435c05c82e2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


## 2. Load and prepare the dataset

we will use the [samsum](https://huggingface.co/datasets/samsum) dataset, a collection of about 16k messenger-like conversations with summaries. Conversations were created and written down by linguists fluent in English.

```python
{
  "id": "13818513",
  "summary": "Amanda baked cookies and will bring Jerry some tomorrow.",
  "dialogue": "Amanda: I baked cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"
}
```

To load the `samsum` dataset, we use the **`load_dataset()`** method from the 🤗 Datasets library.

In [76]:
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets import DatasetDict


data = load_dataset("nightingal3/fig-qa")

def format_for_t5(example):
    """This function will be applied to each example in our datasets."""
    example['input_text'] = f"choose best ending for: {example['startphrase']}"
    example['target_text_ending1'] = example['ending1']
    example['target_text_ending2'] = example['ending2']
    return example

# Assuming `data` is your DatasetDict object
data = data.map(format_for_t5)



In [69]:
data['train'][0]

{'startphrase': 'Her word had the strength of titanium.',
 'ending1': 'Her promises can be believed.',
 'ending2': 'Her promises cannot be trusted.',
 'labels': 0,
 'valid': 1,
 'input_text': 'choose best ending for: Her word had the strength of titanium.',
 'target_text_ending1': 'Her promises can be believed.',
 'target_text_ending2': 'Her promises cannot be trusted.'}

In [77]:
model_id="t5-base"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize_for_t5(example, tokenizer, max_length=512):
    """This function will be applied to each example for tokenization."""
    input_data = tokenizer(example['input_text'], return_tensors="pt", max_length=max_length, truncation=True)
    ending1_data = tokenizer(example['target_text_ending1'], return_tensors="pt", max_length=max_length, truncation=True)
    ending2_data = tokenizer(example['target_text_ending2'], return_tensors="pt", max_length=max_length, truncation=True)
    
    example['input_ids'] = input_data.input_ids.squeeze()
    example['target_ids_ending1'] = ending1_data.input_ids.squeeze()
    example['target_ids_ending2'] = ending2_data.input_ids.squeeze()
    
    if 'target_text_ending1' not in example:
        print(example)
        print(f"Tokenization error for input: {example['input_text']}")

    return example

# Tokenize all the splits in the DatasetDict
for split in ['train', 'validation', 'test']:
    data[split] = data[split].map(lambda x: tokenize_for_t5(x, tokenizer))



Map:   0%|          | 0/9674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

Map:   0%|          | 0/1146 [00:00<?, ? examples/s]

In [79]:
data['train'][0]

{'startphrase': 'Her word had the strength of titanium.',
 'ending1': 'Her promises can be believed.',
 'ending2': 'Her promises cannot be trusted.',
 'labels': 0,
 'valid': 1,
 'input_text': 'choose best ending for: Her word had the strength of titanium.',
 'target_text_ending1': 'Her promises can be believed.',
 'target_text_ending2': 'Her promises cannot be trusted.',
 'input_ids': [854,
  200,
  7784,
  21,
  10,
  1347,
  1448,
  141,
  8,
  2793,
  13,
  28663,
  5,
  1],
 'target_ids_ending1': [1347, 11236, 54, 36, 6141, 5, 1],
 'target_ids_ending2': [1347, 11236, 1178, 36, 7731, 5, 1]}

In [78]:
for split in ['train', 'validation', 'test']:
    if not all('target_ids_ending1' in ex for ex in data[split]):
        print(f"Missing 'target_ids_ending1' in {split} split.")


In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [52]:
import torch
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer

# Define the margin ranking loss
criterion = torch.nn.MarginRankingLoss(margin=1.0)

def compute_contrastive_loss(model, inputs):
    # Extract necessary input data
    input_ids = inputs["input_ids"]
    ending1_ids = inputs["ending1_ids"]
    ending2_ids = inputs["ending2_ids"]
    labels = inputs["labels"]

    # Get model outputs for both endings
    outputs_ending1 = model(input_ids, labels=ending1_ids)
    outputs_ending2 = model(input_ids, labels=ending2_ids)
    
    # Extract logits for loss computation (You might adjust dimensions based on your data shape)
    logits_ending1 = outputs_ending1.logits.squeeze(-1)
    logits_ending2 = outputs_ending2.logits.squeeze(-1)
    
    # Determine target labels for margin ranking loss
    y = torch.tensor([1 if label == 0 else -1 for label in labels], device=logits_ending1.device)

    # Compute the contrastive loss
    loss = criterion(logits_ending1, logits_ending2, y)

    return loss


In [53]:
from transformers import Trainer

class ContrastiveTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Use the contrastive loss computation we defined earlier
        loss = compute_contrastive_loss(model, inputs)
        
        if return_outputs:
            return loss, None
        return loss


In [85]:
def pad_sequence_to_max_length(sequence, max_length, padding_value=0):
    """Pad the sequence with padding_value to max_length."""
    return sequence + [padding_value] * (max_length - len(sequence))

def contrastive_data_collator(features):
    
    try:
        # Determine the max length for padding
        max_length = max(len(f["input_ids"]) for f in features)
        
        print(features[0])

        # Pad the sequences and convert to tensors
        input_ids = torch.stack([torch.tensor(pad_sequence_to_max_length(f["input_ids"], max_length), dtype=torch.long) for f in features], dim=0)

        max_length_ending1 = max(len(f["target_ids_ending1"]) for f in features)
        ending1_ids = torch.stack([torch.tensor(pad_sequence_to_max_length(f["target_ids_ending1"], max_length_ending1), dtype=torch.long) for f in features], dim=0)

        max_length_ending2 = max(len(f["target_ids_ending2"]) for f in features)
        ending2_ids = torch.stack([torch.tensor(pad_sequence_to_max_length(f["target_ids_ending2"], max_length_ending2), dtype=torch.long) for f in features], dim=0)

        labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
    except KeyError as e:
        print("Error processing the following feature:")
        for f in features:
            if 'target_ids_ending1' not in f or 'target_ids_ending2' not in f:
                print(f)
        raise e

    
    return {
        "input_ids": input_ids, 
        "ending1_ids": ending1_ids, 
        "ending2_ids": ending2_ids, 
        "labels": labels
    }



In [82]:
# Diagnostic check for the first batch
first_batch = [data["train"][i] for i in range(training_args.per_device_train_batch_size)]
collated_batch = contrastive_data_collator(first_batch)


In [83]:
def verify_dataset_keys(dataset):
    missing_keys = 0
    for entry in dataset:
        if 'target_ids_ending1' not in entry or 'target_ids_ending2' not in entry:
            missing_keys += 1
    return missing_keys

missing = verify_dataset_keys(data["train"])
print(f"Number of entries missing required keys in training data: {missing}")


Number of entries missing required keys in training data: 0


In [86]:


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    evaluation_strategy="steps",  # or "epoch"
    eval_steps=100,
    save_total_limit=3,
    # more arguments as necessary...
)

trainer = ContrastiveTrainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    data_collator=contrastive_data_collator,
)

trainer.train()



{'labels': 0, 'input_ids': [854, 200, 7784, 21, 10, 100, 681, 56, 36, 38, 813, 17, 99, 179, 38, 4361, 25649, 1]}
Error processing the following feature:
{'labels': 0, 'input_ids': [854, 200, 7784, 21, 10, 100, 681, 56, 36, 38, 813, 17, 99, 179, 38, 4361, 25649, 1]}
{'labels': 1, 'input_ids': [854, 200, 7784, 21, 10, 37, 3196, 33, 38, 1349, 38, 4496, 8, 2403, 8153, 1]}
{'labels': 1, 'input_ids': [854, 200, 7784, 21, 10, 37, 2785, 130, 38, 2937, 38, 3, 9, 1021, 4940, 17452, 5569, 29, 5984, 21, 8, 166, 97, 1]}
{'labels': 1, 'input_ids': [854, 200, 7784, 21, 10, 2144, 23, 2754, 19, 20910, 116, 34, 31, 7, 16, 8, 1059, 13, 12371, 5842, 14882, 5, 1]}
{'labels': 0, 'input_ids': [854, 200, 7784, 21, 10, 37, 18156, 3342, 77, 30, 160, 8677, 47, 38, 3574, 38, 37, 16713, 30, 160, 16594, 1]}
{'labels': 0, 'input_ids': [854, 200, 7784, 21, 10, 37, 1712, 19, 38, 1835, 38, 71, 18358, 18, 189, 1162, 232, 6816, 3519, 157, 6001, 5, 1]}
{'labels': 0, 'input_ids': [854, 200, 7784, 21, 10, 37, 3202, 6528, 16

KeyError: 'target_ids_ending1'

dataset['validation'].

In [10]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Print out a few examples from the raw dataset
print("Raw Data Examples: ", dataset['validation'][:3])

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Apply preprocessing function to a few examples
dummy = dataset['validation'].select(range(3))
preprocessed_examples = dummy.map(preprocess_function, batched=True, remove_columns=["startphrase", "ending1", "ending2", "labels", "valid"])

# Print out preprocessed examples
print("Preprocessed Data Examples: ", preprocessed_examples)

# Prepare a batch of data
input_ids = torch.tensor([example['input_ids'] for example in preprocessed_examples])
attention_mask = torch.tensor([example['attention_mask'] for example in preprocessed_examples])
labels_correct = torch.tensor([example['labels_correct'] for example in preprocessed_examples])
labels_incorrect = torch.tensor([example['labels_incorrect'] for example in preprocessed_examples])

# Print out a few labels
print("Correct Labels: ", labels_correct)
print("Incorrect Labels: ", labels_incorrect)

# Print out model input
print("Model Input: ", {
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels_correct': labels_correct,
    'labels_incorrect': labels_incorrect
})

# Decoding input_ids
decoded_input_ids = [tokenizer.decode(input_id) for input_id in input_ids]
print("Decoded Input IDs: ", decoded_input_ids)

# Replace -100 with tokenizer.pad_token_id for correct labels
labels_correct_replaced = labels_correct.clone()
labels_correct_replaced[labels_correct == -100] = tokenizer.pad_token_id

# Replace -100 with tokenizer.pad_token_id for incorrect labels
labels_incorrect_replaced = labels_incorrect.clone()
labels_incorrect_replaced[labels_incorrect == -100] = tokenizer.pad_token_id

# Decoding labels
decoded_correct_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels_correct_replaced]
print("Decoded Correct Labels: ", decoded_correct_labels)

decoded_incorrect_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels_incorrect_replaced]
print("Decoded Incorrect Labels: ", decoded_incorrect_labels)


Raw Data Examples:  {'startphrase': ['The girl had the flightiness of a sparrow', 'The girl had the flightiness of a rock', 'It was as peaceful as a church.'], 'ending1': ['The girl was very fickle.', 'The girl was very fickle.', 'It was very peaceful.'], 'ending2': ['The girl was very stable.', 'The girl was very stable.', 'It was full of conflict and danger, not peace.'], 'labels': [0, 1, 0], 'valid': [1, 1, 1]}
Preprocessed Data Examples:  Dataset({
    features: ['input_ids', 'attention_mask', 'labels_correct', 'labels_incorrect'],
    num_rows: 3
})
Correct Labels:  tensor([[   37,  3202,    47,   182,   361, 19376,     5,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [   37,  3202,    

In [7]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# huggingface hub model id

#model_id = "philschmid/flan-t5-xxl-sharded-fp16"
model_id = "google/flan-t5-large"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

In [15]:
subset_test_dataset = dataset['validation'].select(range(500))

In [9]:
out_df, preds, labels = evaluate_model(model, tokenizer, subset_test_dataset, verbose = False, total = 500)

100%|██████████| 500/500 [06:15<00:00,  1.33it/s]


In [10]:
zero_shot_accuracy, conf_matrix_zero_shot =  compute_stats(out_df, preds, labels)

overall accuracy: 
0.608
confusion matrix: 
correct forward 304 wrong forward 196 correct backward 346 wrong_backward 154


In [11]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [12]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

# def evaluate_peft_model(sample,max_target_length=50):
#     # generate summary
#     outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
#     prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
#     # decode eval sample
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
#     labels = tokenizer.decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     return prediction, labels

def evaluate_peft_model(sample, max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Print inputs, predicted summary, and reference summary
    input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
    print(f"Input: {input_text}")
    print(f"Predicted Summary: {prediction}")
    print(f"Reference Summary: {labels}")
    print("="*50)  # prints a separator

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for i,sample in tqdm(enumerate(test_dataset)):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)
    if i == 20:
        break

# compute metric 
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results 
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")


# Rogue1: 50.386161%
# rouge2: 24.842412%
# rougeL: 41.370130%
# rougeLsum: 41.394230%

1it [00:02,  2.40s/it]

Input: The girl had the flightiness of a sparrow
Predicted Summary: He would make the bird flee in a moment.
Reference Summary: The girl was very fickle.


2it [00:02,  1.33s/it]

Input: The girl had the flightiness of a rock
Predicted Summary: She looked very young.
Reference Summary: The girl was very stable.


3it [00:03,  1.09it/s]

...

Input: Peace is a human flying
Predicted Summary: Peace is a human flying
Reference Summary: Peace is impossible


20it [00:17,  1.17it/s]

Input: Peace is a human walking
Predicted Summary: , he is a human walking
Reference Summary: Peace is possible


20it [00:17,  1.12it/s]

Input: Plants are a lullaby
Predicted Summary: plantation
Reference Summary: Plants are calming
Rogue1: 16.559615%
rouge2: 5.292784%
rougeL: 16.643501%
rougeLsum: 16.636180%


## 3. Fine-Tune T5 with LoRA and bnb int-8

In addition to the LoRA technique, we will use [bitsanbytes LLM.int8()](https://huggingface.co/blog/hf-bitsandbytes-integration) to quantize out frozen LLM to int8. This allows us to reduce the needed memory for FLAN-T5 XXL ~4x.  

The first step of our training is to load the model. We are going to use [philschmid/flan-t5-xxl-sharded-fp16](https://huggingface.co/philschmid/flan-t5-xxl-sharded-fp16), which is a sharded version of [google/flan-t5-xxl](https://huggingface.co/google/flan-t5-xxl). The sharding will help us to not run off of memory when loading the model.

Now, we can prepare our model for the LoRA int-8 training using `peft`.

In [12]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config 
lora_config = LoraConfig(
 r=8, 
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 785,509,376 || trainable%: 0.30035236651331837


/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


As you can see, here we are only training 0.16% of the parameters of the model! This huge memory gain will enable us to fine-tune the model without memory issues.

Next is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [13]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training.

In [22]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback

class ContrastiveSeq2SeqTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            decoder_input_ids=inputs["decoder_input_ids"] # this line is added
        )
        logits = outputs.logits
        loss = contrastive_loss(logits, inputs["labels_correct"], inputs["labels_incorrect"])

        return (loss, outputs) if return_outputs else loss


        
        return (loss, outputs) if return_outputs else loss


In [23]:
output_dir="lora-flan-t5-large"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=10,
    learning_rate=0.001,
    max_steps=3600,#5000,# ~ 6 epochs
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",  # Adjust this line to match evaluation_strategy
    save_total_limit=3,  # Optional: the number of total saved models
    report_to="tensorboard",
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",
)

# Create Trainer instance with early stopping callback
trainer = ContrastiveSeq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"].select(range(100)),
    #callbacks=[EarlyStoppingCallback(early_stopping_patience= 7, early_stopping_threshold=0.05)]  # Add early stopping callback here
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# train model
trainer.train()


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

Let's now train our model and run the cells below. Note that for T5, some layers are kept in `float32` for stability purposes.

In [41]:
%load_ext tensorboard
%tensorboard --logdir /kaggle/working/lora-flan-t5-base/logs --port 6007

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 461), started 0:00:12 ago. (Use '!kill 461' to kill it.)

The training took ~10:36:00 and cost `~13.22$` for 10h of training. For comparison a [full fine-tuning on FLAN-T5-XXL](https://www.philschmid.de/fine-tune-flan-t5-deepspeed#3-results--experiments) with the same duration (10h) requires 8x A100 40GBs and costs ~322$. 

We can save our model to use it for inference and evaluate it. We will save it to disk for now, but you could also upload it to the [Hugging Face Hub](https://huggingface.co/docs/hub/main) using the `model.push_to_hub` method.

In [49]:
# Save our LoRA model & tokenizer results
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)
# if you want to save the base model to call
# trainer.model.base_model.save_pretrained(peft_model_id)

('results/tokenizer_config.json',
 'results/special_tokens_map.json',
 'results/spiece.model',
 'results/added_tokens.json',
 'results/tokenizer.json')

Our LoRA checkpoint is only 84MB small and includes all of the learnt knowleddge for samsum.

## 4. Evaluate & run Inference with LoRA FLAN-T5

After the training is done we want to evaluate and test it. The most commonly used metric to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries.

We are going to use `evaluate` library to evaluate the `rogue` score. We can run inference using `PEFT` and `transformers`. For our FLAN-T5 XXL model, we need at least 18GB of GPU memory.

In [51]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc. 
peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

Peft model loaded


In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
# Push the model to your namespace with the name "my-finetuned-bert".
model.push_to_hub("T5-Large-figQA-seq2seq-second-run")

adapter_model.bin:   0%|          | 0.00/9.54M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/davidguzmanp/T5-Large-figQA-seq2seq-second-run/commit/0e902a2e7a690f95ccc9ff0af43990fa759dee17', commit_message='Upload model', commit_description='', oid='0e902a2e7a690f95ccc9ff0af43990fa759dee17', pr_url=None, pr_revision=None, pr_num=None)

Let’s load the dataset again with a random sample to try the summarization.

Nice! our model works! Now, lets take a closer look and evaluate it against the `test` set of processed dataset from `samsum`. Therefore we need to use and create some utilities to generate the summaries and group them together. The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries.

----

In [53]:
out_df, preds, labels = evaluate_model(model, tokenizer, subset_test_dataset, verbose = False, total = 500)
zero_shot_accuracy, conf_matrix_zero_shot =  compute_stats(out_df, preds, labels)

100%|██████████| 500/500 [10:54<00:00,  1.31s/it]

overall accuracy: 
0.54
confusion matrix: 
correct forward 270 wrong forward 230 correct backward 260 wrong_backward 240


In [52]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

# def evaluate_peft_model(sample,max_target_length=50):
#     # generate summary
#     outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
#     prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
#     # decode eval sample
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
#     labels = tokenizer.decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     return prediction, labels

def evaluate_peft_model(sample, max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Print inputs, predicted summary, and reference summary
    input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
    print(f"Input: {input_text}")
    print(f"Predicted Summary: {prediction}")
    print(f"Reference Summary: {labels}")
    print("="*50)  # prints a separator

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for i,sample in tqdm(enumerate(test_dataset)):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)
    if i == 20:
        break

# compute metric 
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results 
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

# Rogue1: 50.386161%
# rouge2: 24.842412%
# rougeL: 41.370130%
# rougeLsum: 41.394230%

1it [00:01,  1.41s/it]

Input: The girl had the flightiness of a sparrow
Predicted Summary: The girl was very flighty
Reference Summary: The girl was very fickle.


2it [00:02,  1.29s/it]

Input: The girl had the flightiness of a rock
Predicted Summary: The girl was very tough
Reference Summary: The girl was very stable.


3it [00:03,  1.15s/it]

...

Input: Peace is a human flying
Predicted Summary: Peace is the outcome of positive actions
Reference Summary: Peace is impossible


20it [00:25,  1.52s/it]

Input: Peace is a human walking
Predicted Summary: Peace makes people happy and enlightened
Reference Summary: Peace is possible


20it [00:26,  1.34s/it]

Input: Plants are a lullaby
Predicted Summary: Plants are boring and disinteresting
Reference Summary: Plants are calming
Rogue1: 57.525219%
rouge2: 33.618604%
rougeL: 57.412059%
rougeLsum: 57.256527%


Our PEFT fine-tuned FLAN-T5-XXL achieved a rogue1 score of `50.38%` on the test dataset. For comparison a [full fine-tuning of flan-t5-base achieved a rouge1 score of 47.23](https://www.philschmid.de/fine-tune-flan-t5). That is a `3%` improvements. 

It is incredible to see that our LoRA checkpoint is only 84MB small and model achieves better performance than a smaller fully fine-tuned model.